# Huawei Solar Data Reader via Home Assistant API

This notebook will guide you through the process of reading data from your Huawei Solar inverter and power meter through the Home Assistant API.

## 1. Environment Setup

First, let's create a virtual environment and install the required packages:

In [ ]:
!python -m venv .venv
!source .venv/bin/activate  # On Windows use: .venv\Scripts\activate
!pip install aiohttp

## 2. Import Required Libraries

Let's import the libraries we'll need:

In [ ]:
import asyncio
import aiohttp
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger("ha_solar_reader")

## 3. Define Configuration

Set your Home Assistant configuration. Replace with your actual values:

In [ ]:
# Home Assistant Configuration
HA_URL = "http://your-ha-url:8123"    # Example: "http://192.168.1.100:8123"
HA_TOKEN = "your-long-lived-token"     # Get this from your Home Assistant profile

## 4. Create the HASolarReader Class

This class will handle all the communication with Home Assistant:

In [ ]:
class HASolarReader:
    def __init__(self, ha_url, token):
        self.ha_url = ha_url.rstrip('/')
        self.headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json",
        }
        
        # Inverter entities
        self.inverter_entities = [
            "sensor.inverter_device_status",        # Device status
            "sensor.inverter_input_power",          # Input power
            "sensor.inverter_active_power",         # Active power
            "sensor.inverter_power_factor",         # Power factor
            "sensor.inverter_efficiency",           # Efficiency
            "sensor.daily_yield_energy",            # Daily yield
            "sensor.accumulated_yield_energy"       # Total yield
        ]
        
        # Power meter entities
        self.power_meter_entities = [
            "sensor.power_meter_active_power",      # Current active power
            "sensor.grid_exported_energy",          # Exported energy
            "sensor.grid_accumulated_energy",       # Consumed energy
            "sensor.active_grid_power_factor"       # Grid power factor
        ]

    async def get_entity_state(self, entity_id):
        """Get the state of an entity"""
        async with aiohttp.ClientSession(headers=self.headers) as session:
            url = f"{self.ha_url}/api/states/{entity_id}"
            try:
                async with session.get(url) as response:
                    if response.status == 200:
                        data = await response.json()
                        return {
                            'state': data['state'],
                            'unit': data['attributes'].get('unit_of_measurement', ''),
                            'friendly_name': data['attributes'].get('friendly_name', entity_id)
                        }
                    else:
                        logger.error(f"Error getting {entity_id}: {response.status}")
                        return None
            except Exception as e:
                logger.error(f"Error in request for {entity_id}: {str(e)}")
                return None

    async def get_all_states(self):
        """Get states for all configured entities"""
        # Get inverter data
        inverter_tasks = [self.get_entity_state(entity) for entity in self.inverter_entities]
        inverter_results = await asyncio.gather(*inverter_tasks)
        inverter_data = dict(zip(self.inverter_entities, inverter_results))
        
        # Get power meter data
        meter_tasks = [self.get_entity_state(entity) for entity in self.power_meter_entities]
        meter_results = await asyncio.gather(*meter_tasks)
        meter_data = dict(zip(self.power_meter_entities, meter_results))
        
        return inverter_data, meter_data

    def print_results(self, inverter_data, meter_data):
        """Print formatted results"""
        print("\n===== Huawei Solar System Data =====")
        print(f"Reading time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        print("\n=== INVERTER ===")
        for entity_id, data in inverter_data.items():
            if data:
                unit = f" {data['unit']}" if data['unit'] else ""
                print(f"{data['friendly_name']}: {data['state']}{unit}")
        
        print("\n=== POWER METER ===")
        for entity_id, data in meter_data.items():
            if data:
                unit = f" {data['unit']}" if data['unit'] else ""
                print(f"{data['friendly_name']}: {data['state']}{unit}")

## 5. Test the Connection

Let's first test if we can connect to Home Assistant:

In [ ]:
async def test_connection():
    """Test connection to Home Assistant"""
    async with aiohttp.ClientSession(headers={"Authorization": f"Bearer {HA_TOKEN}"}) as session:
        url = f"{HA_URL}/api/"  # Base API endpoint
        try:
            async with session.get(url) as response:
                if response.status == 200:
                    print("✅ Successfully connected to Home Assistant!")
                    return True
                else:
                    print(f"❌ Connection failed with status: {response.status}")
                    return False
        except Exception as e:
            print(f"❌ Connection error: {str(e)}")
            return False

# Run the test
await test_connection()

## 6. Read Solar Data

Now let's read the actual data from your solar system:

In [ ]:
async def read_solar_data():
    reader = HASolarReader(HA_URL, HA_TOKEN)
    inverter_data, meter_data = await reader.get_all_states()
    reader.print_results(inverter_data, meter_data)

# Read the data
await read_solar_data()

## 7. Set up Periodic Reading (Optional)

If you want to read data periodically:

In [ ]:
async def periodic_reading(interval_seconds=30):
    reader = HASolarReader(HA_URL, HA_TOKEN)
    while True:
        try:
            inverter_data, meter_data = await reader.get_all_states()
            reader.print_results(inverter_data, meter_data)
            await asyncio.sleep(interval_seconds)
        except KeyboardInterrupt:
            print("\nStopping periodic reading...")
            break
        except Exception as e:
            print(f"Error: {str(e)}")
            await asyncio.sleep(5)

# Uncomment to start periodic reading:
# await periodic_reading(30)  # Read every 30 seconds